<a href="https://colab.research.google.com/github/yu0ki/BERT_Practice/blob/main/Chapter5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# この章では、BERTを使って穴埋めタスクを行う


# ライブラリたち
!pip install transformers==4.5.0 fugashi==1.1.0 ipadic==1.0.0

import numpy as np
import torch
from transformers import BertJapaneseTokenizer, BertForMaskedLM


# ちなみに、BertForMaskedLMは特殊トークン[MASK]に入るトークンを語彙の中から予測するクラス

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 27.8 MB/s 
     |████████████████████████████████| 486 kB 70.0 MB/s 
     |████████████████████████████████| 13.4 MB 68.6 MB/s 
     |████████████████████████████████| 880 kB 56.9 MB/s 
     |████████████████████████████████| 3.3 MB 65.8 MB/s 
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556723 sha256=a1142dd56f282035a4924c8f30fcfedfe8d0d9684a52b955e0f4ca6930f98fc7
  Stored in directory: /root/.cache/pip/wheels/33/8b/99/cf0d27191876637cd3639a560f93aa982d7855ce826c94348b
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=5a94d8eed8e7375a8764be487afdd253afec618292cf077bebb02d64fb3210e8
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built ipadic sacremoses


In [3]:
# まずはトークナイザを準備

model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)


# 次は穴埋めタスク用の事前学習済みモデルを準備
bert_mlm = BertForMaskedLM.from_pretrained(model_name)
# GPUにのっける
bert_mlm = bert_mlm.cuda()


Downloading:   0%|          | 0.00/258k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/110 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/479 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
# 「今日は[MASK]へ行く。」　を穴埋めしてみよう

# ・・・とその前に、まずは文章をトークン化したものを見てみよう（[MASK]がちゃんとトークンと見做されている）
text = "今日は[MASK]へ行く。"
tokens = tokenizer.tokenize(text)
print(tokens)


# 手順1：トークン列をトークンIDで置き換える(符号化)
input_ids = tokenizer.encode(text, return_tensors = 'pt')
print(input_ids)

# そしてGPUへ送り込む
input_ids = input_ids.cuda()


['今日', 'は', '[MASK]', 'へ', '行く', '。']
tensor([[   2, 3246,    9,    4,  118, 3488,    8,    3]])


In [5]:
# 手順２：BERTに入力して分類スコアを得る
# １文しか入力してないので、input_ids以外の指定（トークンの最大数とか）が必要ない
with torch.no_grad():
  output = bert_mlm(input_ids = input_ids)
# print(output)

# outputの属性のうち「logits」が、語彙中の各単語に対する分類スコアである
# scoresは、三次元配列：各次元数（サイズ）は(バッチサイズ, 系列長, 語彙のサイズ)
# scores[i, j, k] = 入力された文章のi文目に対応するトークン列の、j番目のトークンに対して、トークンIDがkの語彙のスコア
scores = output.logits
print(scores)

tensor([[[ -5.8525,   5.0457,  -1.7965,  ...,  -4.8386,  -6.4219,  -7.8085],
         [ -4.0218,   7.2845,  -5.3993,  ...,  -6.0369,  -6.5811,  -2.1289],
         [ -5.8364,   5.3641,  -2.2106,  ...,  -4.3529,  -5.7284,  -4.3889],
         ...,
         [ -7.8698,   5.9753,  -4.3922,  ...,  -4.3223,  -6.0900, -11.4386],
         [ -5.4500,   6.5491,   0.0368,  ...,  -4.5615,  -5.1636,  -7.0161],
         [ -8.7510,   3.2686,  -1.6596,  ...,  -5.0593,  -7.0547, -10.7624]]],
       device='cuda:0')


In [6]:
# ちなみにBertForMaskedMLは
# 入力 -> BertModelに入力を入れた時の出力 -> それを線形変換 -> GELU関数（活性化関数） -> 線形変換 -> 最終出力

In [7]:
# 手順３：scoresから[MASK]に入るトークンを予測

# まず、入力された文章（or　文章集合）から、[MASK]（こいつのトークンIDは4）の位置（配列のインデックス）を求める
# input_ids[i].tolist().index(4) : i文目の中でID4に対応するインデックス
mask_position = input_ids[0].tolist().index(4)

# スコアが最も良いトークンのIDを取り出す
# argmax：配列で、一番大きい要素の「インデックス（順番）」を返す関数。括弧の中は初期値（省略可能）
id_best = scores[0, mask_position].argmax(-1).item()

# id_bestに対応するトークンを入手
token_best = tokenizer.convert_ids_to_tokens(id_best)

# 取り出したトークンに「##」がついていた場合(Chapter4参照)は、それを取り除く
token_best = token_best.replace("##", "")

# 元の入力文章の{MASK}を、token_bestで置き換える
final_text = text.replace("[MASK]", token_best)
print(final_text)

今日は東京へ行く。


In [37]:
# 最上位１位だけでなく、上位１０位を求めてみよう

# まずは、text, tokenizer, bert_mlm, num_topk(=上位k件)を入力として、上位num_topk件の穴埋め予測を出す関数を定義
def predict_mask_topk(text, tokenizer, bert_mlm, num_topk):

  # テキストを符号化
  input_ids = tokenizer.encode(text, return_tensors='pt')
  input_ids = input_ids.cuda()



  # bert_mlmに入力（計算結果を保存しないことで、リソースを節約）
  with torch.no_grad():
    output = bert_mlm(input_ids = input_ids)
  # 分類スコアを取得
  scores = output.logits




  # トークンID ４　に対応する、input_idsのインデックスを求める
  # もし複数の[MASK]が含まれていた場合には、 一番最初に見つけたindexを返す
  mask_position = input_ids[0].tolist().index(4)

  # scoresから上位num_topk件を取得
  # topk(n) は上位n件を取得してくれる
  scores_topk = scores[0, mask_position].topk(num_topk)

  # scores_topkのスコアを持つトークンのID列
  # indices はnumpy が提供する関数っぽい。
  # scores_topkのscores[0, masked_position]内でのindex (=token id)を取得
  ids_topk = scores_topk.indices

  # ids_topkを対応するトークンへ変換
  tokens_topk = tokenizer.convert_ids_to_tokens(ids_topk)

  





  # 以上で求めた上位トークンで文中の[MASK]を置き換える
  text_topk = []
  for token in tokens_topk:
    token = token.replace('##', '')

    # [MASK]のうち先頭から1個を置換
    text_topk.append(text.replace('[MASK]', token, 1))

  return text_topk, scores_topk



In [39]:
# 上記の関数で、上位１０件の文章を出力してみよう

text_topk, _ = predict_mask_topk (text, tokenizer, bert_mlm, 10)

# * : 配列を展開
#  sep : 区切り方指定
# option + ¥ でバックスラッシュを打てる
print(*text_topk, sep='\n')

今日は東京へ行く。
今日はハワイへ行く。
今日は学校へ行く。
今日はニューヨークへ行く。
今日はどこへ行く。
今日は空港へ行く。
今日はアメリカへ行く。
今日は病院へ行く。
今日はそこへ行く。
今日はロンドンへ行く。


In [40]:
# MASK箇所が複数ある場合

# 例えば「今日は[MASK][MASK]へ行く。」の穴埋め
# [MASK] の組み合わせは語彙サイズ（=32000）の２乗でめっちゃ計算量やばい

# 近似的な方法を用いよう！　
# 貪欲法
# 一番最初の[MASK]を最も高いスコアを持つトークンで穴埋めする -> 穴埋め後の文章に対して次の[MASK]を最も高いスコアを持つトークンで穴埋めする -> ......


def greedy_prediction (text, tokenizer, bert_mlm):

  for a in range(text.count('[MASK]')):
    # print(text.count("[MASK]"))
    # print(a)
    # print(text)
    text= predict_mask_topk(text, tokenizer, bert_mlm, 1)[0][0]
    
  return text

In [41]:
print(tokenizer.tokenize('今日は[MASK][MASK]へ行く。'))

# 実際穴を埋めてみる
greedy_prediction("今日は[MASK][MASK]へ行く。", tokenizer, bert_mlm)

['今日', 'は', '[MASK]', '[MASK]', 'へ', '行く', '。']


'今日は、東京へ行く。'

In [42]:
# MASKを増やしすぎるとBERTは意味のある文を生成できない。
# なぜなら、BERTは事前学習で文章のうちごく一部のトークンをMASKに置き換えて、周りの文脈から予測するタスクしかしていないから

greedy_prediction("今日は[MASK][MASK][MASK][MASK][MASK]", tokenizer, bert_mlm)

'今日は社会社会的な地位'

In [54]:
# 貪欲法より有能な近似：ビームサーチ
# 貪欲法と違って、最終的な合計スコアが最大になるように狙う

# 特徴
# 複数の文章を同時に処理できる
# 貪欲法と違い、最終結果が最高スコアになるように

# 手順
# 複数の文章が与えられたとする
# １。一つ目の[MASK]に対して、上位１０件分の解候補を出し、穴を埋める
# ２。得られた10個の文それぞれに対して、２つ目の[MASK]に対する10個の解候補を出す
# ３。得られた１００通りの文章の中から、スコアの良い10個を選び他は捨てる
# この流れを繰り返す


def beam_search(text, tokenizer, bert_mlm, num_topk):

  # マスクの数
  num_mask = text.count('[MASK]')

  # ループ中で使用する配列の初期化
  text_topk = [text]
  scores_topk = np.array([0])


  # maskの数だけ以下を繰り返す
  for _ in range(num_mask):

    # 解候補(num_topkの２乗個出てくるはず)は以下に格納
    text_candidates = []
    score_candidates = []

    # ここまでに得られているtext, scoreの上位num_topk件から、新たな解候補を作成
    for text_mask, score in zip(text_topk, scores_topk):
      text_cand, score_cand = predict_mask_topk(text_mask, tokenizer, bert_mlm, num_topk)
      text_candidates.extend(text_cand)

      # TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.
      # を回避するためにscore_candに色々つけた。教科書では、predict_mask_topkの方で、スコアに関する出力にこれをつけてくれている
      score_candidates.append(score + score_cand.values.cpu().numpy())

    
    # それらから上位num_topk個を残す
    # hstackは、与えられた複数の配列を全て結合する。つまり１次元配列に直されている
    score_candidates = np.hstack(score_candidates)

    # np.argsort()は値ではなく並び替えたインデックス（元のndarrayでの位置 = 0始まりの順番）のndarrayを返す
    # 昇順ソートなので、[::-1]で逆順にする
    # [:num_topk]で上位候補を絞る
    index_list = score_candidates.argsort()[::-1][:num_topk]

    # 該当候補のtextの配列
    text_topk = [text_candidates[idx] for idx in index_list]
    # さらに対応するスコア
    score_topk = score_candidates[index_list]

  return text_topk

In [55]:
text = "今日は[MASK][MASK]へ行く。"
text_topk = beam_search(text, tokenizer, bert_mlm, 10)
print(*text_topk, sep='\n')

今日は、東京へ行く。
今日は、ハワイへ行く。
今日は、学校へ行く。
今日は、ニューヨークへ行く。
今日は、空港へ行く。
今日は、北海道へ行く。
今日は、パリへ行く。
今日は、アメリカへ行く。
今日は、日本へ行く。
今日は、病院へ行く。


In [61]:
# ちなみに実験すると
# MASKを増やしすぎるとBERTは意味のある文を生成できない。
# なぜなら、BERTは事前学習で文章のうちごく一部のトークンをMASKに置き換えて、周りの文脈から予測するタスクしかしていないから

print(*beam_search("今日は[MASK][MASK][MASK][MASK][MASK]", tokenizer, bert_mlm, 10), sep='\n')

今日は社会社会的な地位
今日は社会社会的な組織
今日は社会社会的なもの
今日は社会社会的な活動
今日は社会社会的な団体
今日は社会社会的な状況
今日は社会社会的な概念
今日は社会社会的な役割
今日は社会社会的な存在
今日は社会社会的な意味
